In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

In [ ]:
def load_csvs(folder):
    data_path = f"C:\\path\\to\\expts\\{folder}"
    global_L1 = pd.read_csv(f"{data_path}\\super_res_validation.csv", index_col=0, header=[0, 1])

    if folder != "multi":
        global_L1 = global_L1[[c for c in global_L1.columns if len(c[1].split('-')) < 3]]

    return global_L1

In [ ]:
sr_cols = [f"sr_vq{2**i}" for i in range(11)]
jo_cols = [f"jo_vq{2**i}" for i in range(11)]

def reorder_dfs(global_df):
    global_df = global_df.loc[:, pd.IndexSlice[:, ["sr_vq0"] + sr_cols + jo_cols]]
    return global_df

In [ ]:
deep = load_csvs("deep")
deep = reorder_dfs(deep)

shallow = load_csvs("shallow")
shallow = reorder_dfs(shallow)

multi = load_csvs("multi")
multi.rename({"ce_vqda1": "ce_vq128-128-512-1024-64", "jo_vqda1": "jo_vq128-128-256-512-512"}, axis=1, level=1, inplace=True)

In [ ]:
idx = shallow.columns.get_level_values(1).unique()
shallow.median().unstack().T.loc[idx]  # sr_vq64

In [ ]:
def plot_boxplots(global_df):
    plt.figure(figsize=(18, 8))
    plt.subplot(2, 1, 1)
    plt.boxplot(global_df["L1"])
    plt.xticks(ticks=np.arange(1, global_df["L1"].shape[1] + 1), labels=global_df["L1"].columns, rotation="vertical")
    plt.subplot(2, 1, 2)
    plt.boxplot(global_df["focal_L1"])
    plt.xticks(ticks=np.arange(1, global_df["L1"].shape[1] + 1), labels=global_df["L1"].columns, rotation="vertical")
    plt.tight_layout()
    plt.show()

In [ ]:
g_df = shallow
df_sr_cols = [c for c in g_df["L1"].columns if "sr" in c and c != "sr_vq0"]
df_jo_cols = [c for c in g_df["L1"].columns if "jo" in c]

plot_boxplots(g_df.loc[:, pd.IndexSlice[:, df_sr_cols]])
print(f"Global: {stats.kruskal(*g_df['L1'][df_sr_cols].values.T.tolist())}")
print(f"Focal: {stats.kruskal(*g_df['focal_L1'][df_sr_cols].values.T.tolist())}")

In [ ]:
q = 0.5
multi["focal_L1"].quantile([0.05, 0.5, 0.95]).T#.rank()